# Projet CoCoMa - Coordination de satellites
#### Tom Bouscarat et Chanattan Sok

### Ce notebook présente notre projet dans un format de présentation qui suit la trame donnée par le sujet.

### Préparation : Instances et planification des tâches
#### 1. Générateur aléatoire d'instances

In [1]:
from InstanceGenerator import *

In [2]:
nb_satellites = 3
nb_users = 3
nb_tasks = 5
seed = None
inst = generate_ESOP_instance(nb_satellites=nb_satellites, nb_users=nb_users, nb_tasks=nb_tasks, seed=seed)
print(inst.to_text())

[Parameters]
Satellites : 3
Exclusive users : 3
Tasks : 5

[Satellites]
s0 0 300 20 1
s1 0 300 20 1
s2 0 300 20 1

[Users]
u0 -
u1 s1:124-176, s1:162-183
u2 s0:40-61, s2:5-40
u3 s1:86-140, s0:68-114

[Tasks]
r_0 owner=u1 window=[7,202] duration=9 reward=5
r_1 owner=u2 window=[119,217] duration=7 reward=2
r_2 owner=u2 window=[75,265] duration=5 reward=1
r_3 owner=u2 window=[103,150] duration=3 reward=6
r_4 owner=u2 window=[128,140] duration=3 reward=2

[Observations]
o_r_0_0 task=r_0 owner=u1 sat=s0 window=[167,182] duration=9 reward=5
o_r_0_1 task=r_0 owner=u1 sat=s0 window=[60,124] duration=9 reward=5
o_r_0_2 task=r_0 owner=u1 sat=s1 window=[128,163] duration=9 reward=5
o_r_0_3 task=r_0 owner=u1 sat=s0 window=[43,119] duration=9 reward=5
o_r_1_0 task=r_1 owner=u2 sat=s1 window=[127,205] duration=7 reward=2
o_r_1_1 task=r_1 owner=u2 sat=s0 window=[142,190] duration=7 reward=2
o_r_1_2 task=r_1 owner=u2 sat=s2 window=[165,217] duration=7 reward=2
o_r_2_0 task=r_2 owner=u2 sat=s1 window=[

#### 2. Méthode de planification pour un utilisateur : Greedy Solver 

In [3]:
from GreedySolver import *

##### Planification locale pour l'utilisateur u0 et affichage de son plan.

In [4]:
plan = greedy_schedule_for_user(inst, user_id='u0')
print_user_plans({'u0': plan})

Planning pour l'utilisateur u0:
  Sur le satellite s0:
  Sur le satellite s1:
  Sur le satellite s2:
  Score total: 0



##### Temps de calcul pour greedy_schedule_for_user.

In [5]:
# Moyenne de temps d'exécution pour greedy_schedule_for_user pour différentes instances ESOP.
# Vérification de l'optimalité : TODO.
import time
import numpy as np
exec_times = []

for i in range(10):
    inst = generate_ESOP_instance(nb_satellites=3 + np.random.randint(5, 10), nb_users=5 + np.random.randint(5, 10), nb_tasks=50 + np.random.randint(0, 100), seed=None)
    start_time = time.time()
    plan = greedy_schedule_for_user(inst, user_id='u0')
    end_time = time.time()
    elapsed_time = end_time - start_time
    exec_times.append(elapsed_time)
print(f"Temps d'exécution moyen pour greedy_schedule_for_user: {sum(exec_times)/len(exec_times):.6f} secondes") # ordre de 10^-4 secondes.

Temps d'exécution moyen pour greedy_schedule_for_user: 0.001182 secondes


## Partie 1 : Optimisation de contraintes distribuées
#### 1. Génération du DCOP associé à une instance ESOP.

In [6]:
from DCOP import *

In [7]:
inst2 = generate_ESOP_instance(nb_satellites=3, nb_users=3, nb_tasks=20, seed=42)
dcop = generate_DCOP_instance(inst2)
print(dcop)

name: esop_dcop
objective: min
domains:
  binary:
    values:
    - 0
    - 1
agents:
- u1
- u2
- u3
- aux_3
- aux_4
- aux_5
- aux_6
- aux_7
- aux_8
- aux_9
- aux_10
- aux_11
- aux_12
- aux_13
- aux_14
- aux_15
- aux_16
- aux_17
- aux_18
- aux_19
- aux_20
- aux_21
- aux_22
- aux_23
- aux_24
- aux_25
- aux_26
- aux_27
- aux_28
- aux_29
- aux_30
- aux_31
- aux_32
- aux_33
- aux_34
- aux_35
- aux_36
- aux_37
- aux_38
- aux_39
- aux_40
- aux_41
- aux_42
- aux_43
- aux_44
- aux_45
- aux_46
- aux_47
- aux_48
- aux_49
- aux_50
- aux_51
- aux_52
- aux_53
- aux_54
- aux_55
- aux_56
- aux_57
- aux_58
- aux_59
- aux_60
- aux_61
- aux_62
- aux_63
- aux_64
- aux_65
- aux_66
- aux_67
- aux_68
- aux_69
- aux_70
- aux_71
- aux_72
- aux_73
- aux_74
- aux_75
- aux_76
- aux_77
- aux_78
- aux_79
- aux_80
- aux_81
- aux_82
- aux_83
- aux_84
- aux_85
- aux_86
- aux_87
- aux_88
- aux_89
- aux_90
- aux_91
- aux_92
- aux_93
- aux_94
- aux_95
- aux_96
- aux_97
- aux_98
- aux_99
- aux_100
- aux_101
- aux_102
- a

#### 2. Résolution du DCOP

In [8]:
res = solve_dcop(inst2) # L'instance inst2 est directement transformée en DCOP. NB : est nécessaire PyDcop installé et accessible dans l'environnement.
print(res)


=== Résolution DCOP avec DPOP ===

> Génération du DCOP...
Validation des fonctions de contraintes...
Toutes les fonctions sont valides
> DCOP sauvegardé dans esop_dcop.yaml

> Informations du DCOP:
  - Nombre de variables: 45
  - Nombre de contraintes: 79

Lancement de DPOP...
Temps de résolution DPOP : 16.2268 secondes

> Sortie de DPOP:
--------------------------------------------------
{
  "agt_metrics": {
    "aux_10": {
      "activity_ratio": 0.0046863252567194025,
      "count_ext_msg": {
        "_discovery_aux_10": 18,
        "x_u3_o_r_8_0": 2
      },
      "cycles": {
        "x_u3_o_r_8_0": 0
      },
      "size_ext_msg": {
        "_discovery_aux_10": 0,
        "x_u3_o_r_8_0": 274
      }
    },
    "aux_100": {
      "activity_ratio": 4.730044953209465e-07,
      "count_ext_msg": {
        "_discovery_aux_100": 3
      },
      "cycles": {},
      "size_ext_msg": {
        "_discovery_aux_100": 0
      }
    },
    "aux_101": {
      "activity_ratio": 4.6445636989572

#### 3. Comparaison expérimentale (Centralized Greedy Solver vs SDCOP)

In [9]:
from Main import *
extensive_compare_greedy_vs_sdcop()


=== INSTANCE 1 / 10 ===


=== INSTANCE 2 / 10 ===

Erreur lors de l'exécution de pydcop: Command '['pydcop', 'solve', '--algo', 'dpop', 'sdcop_req_r_37.yaml']' returned non-zero exit status 1.
Stderr: Traceback (most recent call last):
  File "/Users/csok/.pyenv/versions/pydcop/bin/dcop_cli.py", line 212, in <module>
    main()
  File "/Users/csok/.pyenv/versions/pydcop/bin/dcop_cli.py", line 140, in main
    args.func(args)
  File "/Users/csok/.pyenv/versions/3.8.16/envs/pydcop/lib/python3.8/site-packages/pydcop/commands/solve.py", line 486, in run_cmd
    distribution = dist_module.distribute(
  File "/Users/csok/.pyenv/versions/3.8.16/envs/pydcop/lib/python3.8/site-packages/pydcop/distribution/oneagent.py", line 128, in distribute
    raise ImpossibleDistributionException(
pydcop.distribution.objects.ImpossibleDistributionException: Not enough agents for one agent for each computation : 4 < 5


=== INSTANCE 3 / 10 ===


=== INSTANCE 4 / 10 ===

Erreur lors de l'exécution de pydcop: